In [5]:
import numpy as np
import networkx as nx
from numpy import genfromtxt
import matplotlib.pyplot as plt
import sys
import random 


In [6]:
def generate_path(list_of_locations, starting_car_location, adjacency_matrix):
    path = []
    
    starting_index = list_of_locations.index(starting_car_location)
    next_index = adjacency_matrix[starting_index].index(random.choice([i for i in adjacency_matrix[starting_index] if i != 0]))
    path.append(starting_index)
    path.append(next_index)
    while next_index != starting_index:
        next_index = adjacency_matrix[next_index].index(random.choice([i for i in adjacency_matrix[next_index] if i != 0]))
        path.append(next_index)
        
    return path

def generate_drop_off_locations(path):
    path_length = len(path)
    # pick a random number of locations as drop-off locations
    drop_off_locations = random.sample(path, random.randrange(1, path_length - 1))
    return drop_off_locations

def find_closest_dropoff_location_to_ta(drop_off_locations, list_of_locations, list_of_homes, adjacency_matrix):
    num_vertices = len(adjacency_matrix[0])
    
    # filling in a dictionary mapping TA home to [dropoff location, dropoff distance]
    dict_ta_to_dropoff_location = {}
    for h in list_of_homes:
        dict_ta_to_dropoff_location[list_of_locations.index(h)] = [0, sys.maxsize]
    
    # finding the closest dropoff location to each TA's home using previous dictionary dict_ta_to_dropoff_location
    for d in drop_off_locations:
        for h in list_of_homes:
            home_index = list_of_locations.index(h)
            dist_d_to_h = dijkstra(num_vertices, adjacency_matrix, d)[home_index]
            if dist_d_to_h < dict_ta_to_dropoff_location[home_index][1]:
                dict_ta_to_dropoff_location[home_index] = [d, dist_d_to_h]
    
    # converting dict_ta_to_dropoff_location to output dictionary format
    ret_dict = {}  # dict mapping dropoff_location to list_of_TAs
    for h in dict_ta_to_dropoff_location:
        if dict_ta_to_dropoff_location[h][0] in ret_dict:
            ret_dict[dict_ta_to_dropoff_location[h][0]].append(h)
        else:
            ret_dict[dict_ta_to_dropoff_location[h][0]] = [h]
    
    return ret_dict

In [7]:
def minDistance(num_vertices, dist, sptSet):

    min = sys.maxsize

    for v in range(num_vertices):
        if dist[v] < min and sptSet[v] == False:
            min = dist[v]
            min_index = v

    return min_index

# returns array that represents distances from src to each other vertex (by index) in graph 
def dijkstra(num_vertices, adjacency_matrix, src):    
    dist = [sys.maxsize] * num_vertices
    dist[src] = 0
    sptSet = [False] * num_vertices

    for cout in range(num_vertices):
        u = minDistance(num_vertices, dist, sptSet)
        sptSet[u] = True
        
        for v in range(num_vertices):
            if adjacency_matrix[u][v] > 0 and sptSet[v] == False and \
                dist[v] > dist[u] + adjacency_matrix[u][v]:
                dist[v] = dist[u] + adjacency_matrix[u][v]

    return dist

In [8]:
def solve(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix, params=[]):
    
    total_energy = sys.maxsize
    final_path = []
    final_ret_dict = {}
    
    # 100 random iterations
    for i in range(10000):

        # generating a random path first (Can be optimized by choosing paths from MST and using DFS)
        path = generate_path(list_of_locations, starting_car_location, adjacency_matrix)

        # generate random drop off locations
        drop_off_locations = generate_drop_off_locations(path)
        
        # generate dictionary mapping drop off locations to list of ta's
        ret_dict = find_closest_dropoff_location_to_ta(drop_off_locations, list_of_locations, list_of_homes, adjacency_matrix)
    
        curr_total_energy = 0
    
        # finding total_energy from just driving
        for p1 in range(len(path) - 1):
            curr_total_energy += 0.667 * adjacency_matrix[path[p1]][path[p1+1]]
    
        # finding total_energy from ta's walking as well
        for k in ret_dict:
            for v in ret_dict[k]:
                curr_total_energy += adjacency_matrix[k][v]
                
        if curr_total_energy < total_energy:
            # update path and ret_dict
            total_energy = curr_total_energy
            final_path = path
            final_ret_dict = ret_dict
        
    
    return final_path, final_ret_dict, total_energy
    

solve(["Soda", "location1", "location2", "location3"],
      ["location2", "location1"],
      "Soda",
      [[0, 9, 7, 5],
       [9, 0, 3, 7],
       [7, 3, 0, 5],
       [5, 7, 5, 6]])

([0, 2, 0], {2: [2, 1]}, 12.338000000000001)